In [3]:
import pandas as pd


results_df = pd.DataFrame(
    {
        'Model': [],
        'Accuracy': [],
        'Recall': [],
        'ROC-AUC': [],
        'PR-AUC': [],
    }
).astype(
    {
        'Model': str,
        'Accuracy': float,
        'Recall': float,
        'ROC-AUC': float,
        'PR-AUC': float,
    }
)

In [4]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf
)

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


pipeline_knn = ImbPipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

param_grid_knn = {
    'knn__n_neighbors': [3, 5, 7, 11, 15, 21, 31, 51],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    estimator=pipeline_knn,
    param_grid=param_grid_knn,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1,
)

grid_knn.fit(X, y)

print('KNN best params:', grid_knn.best_params_)
print('KNN best recall:', grid_knn.best_score_)

results_df = evaluate_and_append(
    model_name='KNeighborsClassifier',
    best_estimator=grid_knn.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 32 candidates, totalling 160 fits
KNN best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 3, 'knn__weights': 'distance'}
KNN best recall: 0.053342816500711245
                  Model  Accuracy    Recall   ROC-AUC   PR-AUC
0  KNeighborsClassifier  0.933699  0.053476  0.565712  0.06636


In [5]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf
)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


dt = DecisionTreeClassifier(class_weight='balanced', random_state=42)

param_grid_dt = {
    'max_depth': [6, 8, 10, 12, 15],
    'min_samples_leaf': [1, 2, 5, 10],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

grid_dt = GridSearchCV(
    estimator=dt,
    param_grid=param_grid_dt,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

grid_dt.fit(X, y)

print('DecisionTree best params:', grid_dt.best_params_)
print('DecisionTree recall:', grid_dt.best_score_)

results_df = evaluate_and_append(
    model_name='DecisionTreeClassifier',
    best_estimator=grid_dt.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
DecisionTree best params: {'criterion': 'gini', 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'min_samples_split': 2}
DecisionTree recall: 0.8660028449502134
                    Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier  0.933699  0.053476  0.565712  0.066360
1  DecisionTreeClassifier  0.621248  0.866310  0.774375  0.123078


In [6]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf
)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


rf = RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
    n_jobs=-1
)

param_grid_rf = {
    'n_estimators': [300, 500],
    'max_depth': [8, 12, None],
    'min_samples_leaf': [2, 5],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

grid_rf.fit(X, y)

print('RandomForest best params:', grid_rf.best_params_)
print('RandomForest best recall:', grid_rf.best_score_)

results_df = evaluate_and_append(
    model_name='RandomForestClassifier',
    best_estimator=grid_rf.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
RandomForest best params: {'criterion': 'entropy', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 300}
RandomForest best recall: 0.5766714082503556
                    Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier  0.933699  0.053476  0.565712  0.066360
1  DecisionTreeClassifier  0.621248  0.866310  0.774375  0.123078
2  RandomForestClassifier  0.818585  0.577540  0.815964  0.147802


In [7]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf,
    CAT_FEATURES
)

from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV


cb = CatBoostClassifier(
    random_state=42,
    thread_count=-1,
    verbose=False,
    od_type='Iter',
    od_wait=50,
    eval_metric='Recall',
    auto_class_weights='Balanced',
    cat_features=CAT_FEATURES
)

param_grid_cb = {
    'iterations': [1000],
    'learning_rate': [0.03, 0.06, 0.1],
    'depth': [6, 8],
    'l2_leaf_reg': [3, 7]
}

grid_cb = GridSearchCV(
    estimator=cb,
    param_grid=param_grid_cb,
    cv=skf,
    scoring='recall',
    n_jobs=1,
    verbose=1
)

grid_cb.fit(X, y)

print('CatBoost best params:', grid_cb.best_params_)
print('CatBoost best recall:', grid_cb.best_score_)

results_df = evaluate_and_append(
    model_name='CatBoostClassifier',
    best_estimator=grid_cb.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
CatBoost best params: {'depth': 6, 'iterations': 1000, 'l2_leaf_reg': 7, 'learning_rate': 0.03}
CatBoost best recall: 0.23556187766714082
                    Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier  0.933699  0.053476  0.565712  0.066360
1  DecisionTreeClassifier  0.621248  0.866310  0.774375  0.123078
2  RandomForestClassifier  0.818585  0.577540  0.815964  0.147802
3      CatBoostClassifier  0.900809  0.235294  0.787578  0.135529


In [8]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf
)

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV


neg_count = (y==0).sum()
pos_count = (y==1).sum()
scale_pos_weight_value = neg_count/pos_count if pos_count > 0 else 1

xgb = XGBClassifier(
    random_state=42,
    n_jobs=-1,
    verbosity=0,
    eval_metric='aucpr',
    tree_method='hist',
    scale_pos_weight=scale_pos_weight_value
)

param_grid_xgb = {
    'n_estimators': [1000],
    'learning_rate': [0.03, 0.06, 0.1],
    'max_depth': [6, 8],
    'reg_lambda': [3, 7],
    'min_child_weight': [1, 3],
    'gamma': [0, 0.1, 0.3]
}

grid_xgb = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid_xgb,
    cv=skf,
    scoring='recall',
    n_jobs=1,
    verbose=1
)

grid_xgb.fit(X, y)

print('XGBoost best params:', grid_xgb.best_params_)
print('XGBoost best recall:', grid_xgb.best_score_)

results_df = evaluate_and_append(
    model_name='XGBoostClassifier',
    best_estimator=grid_xgb.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
XGBoost best params: {'gamma': 0.3, 'learning_rate': 0.06, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 1000, 'reg_lambda': 7}
XGBoost best recall: 0.31024182076813656
                    Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier  0.933699  0.053476  0.565712  0.066360
1  DecisionTreeClassifier  0.621248  0.866310  0.774375  0.123078
2  RandomForestClassifier  0.818585  0.577540  0.815964  0.147802
3      CatBoostClassifier  0.900809  0.235294  0.787578  0.135529
4       XGBoostClassifier  0.895328  0.310160  0.804546  0.151915


In [ ]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf
)

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


pipeline_mlp = ImbPipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(
        random_state=42,
        max_iter=1000,
        early_stopping=True,
        n_iter_no_change=50,
        validation_fraction=0.1,
        verbose=False
    ))
])

param_grid_mlp = {
    'mlp__hidden_layer_sizes': [(100,), (150,), (100, 50)],
    'mlp__activation': ['relu', 'tanh'],
    'mlp__solver': ['adam', 'lbfgs'],
    'mlp__alpha': [0.001, 0.01],
    'mlp__learning_rate_init': [0.001, 0.005],
    'mlp__batch_size': ['auto', 256]
}

grid_mlp = GridSearchCV(
    estimator=pipeline_mlp,
    param_grid=param_grid_mlp,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1
)

grid_mlp.fit(X, y)

print('MLP best params:', grid_mlp.best_params_)
print('MLP best recall:', grid_mlp.best_score_)

results_df = evaluate_and_append(
    model_name='MLPClassifier',
    best_estimator=grid_mlp.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
MLP best params: {'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__batch_size': 'auto', 'mlp__hidden_layer_sizes': (100,), 'mlp__learning_rate_init': 0.001, 'mlp__solver': 'lbfgs'}
MLP best recall: 0.12816500711237552
                    Model  Accuracy    Recall   ROC-AUC    PR-AUC
0    KNeighborsClassifier  0.933699  0.053476  0.565712  0.066360
1  DecisionTreeClassifier  0.621248  0.866310  0.774375  0.123078
2  RandomForestClassifier  0.818585  0.577540  0.815964  0.147802
3      CatBoostClassifier  0.900809  0.235294  0.787578  0.135529
4       XGBoostClassifier  0.895328  0.310160  0.804546  0.151915
5           MLPClassifier  0.908640  0.128342  0.689968  0.093368


In [17]:
results_df.to_csv('results_train_orig.csv', index=False)